In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 91427442,
 'calculationPrice': 'close',
 'change': 11.4,
 'changePercent': 0.07671,
 'close': 159.26,
 'closeSource': 'iofiflca',
 'closeTime': 1719769466549,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': 161.568,
 'delayedPriceTime': 1746407455272,
 'extendedChange': 0.11,
 'extendedChangePercent': 0.00073,
 'extendedPrice': 160.26,
 'extendedPriceTime': 1738055441254,
 'high': 159.26,
 'highSource': 'eu1elrepd e ntdmicy5ai ',
 'highTime': 1717761928477,
 'iexAskPrice': None,
 'iexAskSize': None,
 'iexBidPrice': None,
 'iexBidSize': None,
 'iexClose': 160.22,
 'iexCloseTime': 1705763359035,
 'iexLastUpdated': None,
 'iexMarketPercent': None,
 'iexOpen': 150.69,
 'iexOpenTime': 1740753865603,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexVolume': None,
 'lastTradeTime': 1748199113845,
 'latestPrice': 159.27,
 'latestSource': 'Close',
 'latestTime': 'October 28, 2022',
 'latestUpdate': 1674811224254,
 'latestVolume': 168284977,
 'low': 

In [4]:
pe_ratio = data['peRatio']
pe_ratio

26.38

In [15]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [16]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        new_row = pd.Series([symbol, data[symbol]['quote']['latestPrice'], data[symbol]['quote']['peRatio'],  'N/A'],  index = my_columns)
        final_dataframe = pd.concat([final_dataframe, new_row.to_frame().T], ignore_index=True)
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MMM,126.9,11.49,N/A
1,AOS,56.43,17,N/A
2,ABT,104.15,20.69,N/A
3,ABBV,147.94,21.79,N/A
4,ABMD,269.23,56.44,N/A
...,...,...,...,...
497,YUM,121.62,24.29,N/A
498,ZBRA,289,31.45,N/A
499,ZBH,113.47,109.13,N/A
500,ZION,52.19,6.05,N/A


In [17]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,46.66,3.3,N/A
1,AIG,57.5,3.88,N/A
2,LUMN,7.41,3.9,N/A
3,PVH,53.68,4.09,N/A
4,PHM,41.55,4.2,N/A
5,NUE,138,4.31,N/A
6,F,13.73,4.64,N/A
7,MRNA,153.69,4.67,N/A
8,LYB,80.06,4.79,N/A
9,DISH,16,4.83,N/A


In [19]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")
        
portfolio_input()

Enter the value of your portfolio:10000000


In [20]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = position_size // final_dataframe['Price'][i]
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,46.66,3.3,4286
1,AIG,57.5,3.88,3478
2,LUMN,7.41,3.9,26990
3,PVH,53.68,4.09,3725
4,PHM,41.55,4.2,4813
5,NUE,138,4.31,1449
6,F,13.73,4.64,14566
7,MRNA,153.69,4.67,1301
8,LYB,80.06,4.79,2498
9,DISH,16,4.83,12500


In [21]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [23]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN

            
        new_row = pd.Series([symbol, 
                             data[symbol]['quote']['latestPrice'],
                             'N/A',
                             data[symbol]['quote']['peRatio'],
                             'N/A',
                             data[symbol]['advanced-stats']['priceToBook'],
                             'N/A',
                             data[symbol]['advanced-stats']['priceToSales'],
                             'N/A',
                             ev_to_ebitda,
                             'N/A',
                             ev_to_gross_profit,
                             'N/A',
                             'N/A'],
                            index = rv_columns)
        rv_dataframe = pd.concat([rv_dataframe, new_row.to_frame().T], ignore_index=True)

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,127.6,N/A,11.51,N/A,5.16,N/A,2.08,N/A,12.557186,N/A,5.329887,N/A,N/A
1,AOS,55.4,N/A,17.12,N/A,4,N/A,1.85,N/A,9.361305,N/A,5.020433,N/A,N/A
2,ABT,101.55,N/A,21.53,N/A,4.9,N/A,3.93,N/A,14.915742,N/A,7.136027,N/A,N/A
3,ABBV,147.75,N/A,21.23,N/A,18.24,N/A,4.73,N/A,11.590845,N/A,7.677327,N/A,N/A
4,ABMD,263.93,N/A,56.47,N/A,7.77,N/A,11.2,N/A,38.172997,N/A,12.606233,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,120.02,N/A,24.58,N/A,-4.08,N/A,5.13,N/A,20.221361,N/A,9.07244,N/A,N/A
498,ZBRA,297,N/A,31.92,N/A,6.02,N/A,2.58,N/A,15.673768,N/A,6.553558,N/A,N/A
499,ZBH,113.91,N/A,109.59,N/A,2.06,N/A,3.34,N/A,18.48796,N/A,5.717305,N/A,N/A
500,ZION,51.75,N/A,6.08,N/A,1.51,N/A,2.29,N/A,4.752421,N/A,2.218578,N/A,N/A


In [26]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
24,GOOG,99.19,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
43,AON,293.7,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
64,BRK.B,301.8,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
99,CERN,98.07,N/A,50.24,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
114,CTXS,106,N/A,41.1,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
148,DISCK,25.13,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FRT,101.66,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,FOX,27.49,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
260,J,121.54,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
331,NWS,17.71,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [27]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [28]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [29]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.223108
1       0.39243
2      0.503984
3      0.496016
4      0.922311
         ...   
497    0.568725
498    0.760956
499    0.974104
500    0.085657
501    0.796813
Name: PE Percentile, Length: 502, dtype: object
0      0.669323
1      0.562749
2      0.646414
3      0.930279
4      0.786853
         ...   
497    0.057769
498    0.713147
499    0.320717
500    0.201195
501    0.926295
Name: PB Percentile, Length: 502, dtype: object
0      0.370518
1      0.324701
2      0.696215
3       0.75996
4      0.956175
         ...   
497    0.794821
498    0.463147
499    0.595618
500    0.418327
501    0.915339
Name: PS Percentile, Length: 502, dtype: object
0      0.430279
1      0.288845
2      0.561753
3      0.380478
4      0.962151
         ...   
497    0.792829
498    0.605578
499    0.737052
500    0.081673
501    0.908367
Name: EV/EBITDA Percentile, Length: 502, dtype: object
0      0.394422
1      0.366534
2      0.569721
3      0.619522
4      0.864542
         ...   
4

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,127.6,N/A,11.51,0.223108,5.16,0.669323,2.08,0.370518,12.557186,0.430279,5.329887,0.394422,N/A
1,AOS,55.4,N/A,17.12,0.39243,4.00,0.562749,1.85,0.324701,9.361305,0.288845,5.020433,0.366534,N/A
2,ABT,101.55,N/A,21.53,0.503984,4.90,0.646414,3.93,0.696215,14.915742,0.561753,7.136027,0.569721,N/A
3,ABBV,147.75,N/A,21.23,0.496016,18.24,0.930279,4.73,0.75996,11.590845,0.380478,7.677327,0.619522,N/A
4,ABMD,263.93,N/A,56.47,0.922311,7.77,0.786853,11.20,0.956175,38.172997,0.962151,12.606233,0.864542,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,120.02,N/A,24.58,0.568725,-4.08,0.057769,5.13,0.794821,20.221361,0.792829,9.072440,0.695219,N/A
498,ZBRA,297,N/A,31.92,0.760956,6.02,0.713147,2.58,0.463147,15.673768,0.605578,6.553558,0.494024,N/A
499,ZBH,113.91,N/A,109.59,0.974104,2.06,0.320717,3.34,0.595618,18.487960,0.737052,5.717305,0.426295,N/A
500,ZION,51.75,N/A,6.08,0.085657,1.51,0.201195,2.29,0.418327,4.752421,0.081673,2.218578,0.125498,N/A


In [30]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,127.6,N/A,11.51,0.223108,5.16,0.669323,2.08,0.370518,12.557186,0.430279,5.329887,0.394422,0.41753
1,AOS,55.4,N/A,17.12,0.39243,4.00,0.562749,1.85,0.324701,9.361305,0.288845,5.020433,0.366534,0.387052
2,ABT,101.55,N/A,21.53,0.503984,4.90,0.646414,3.93,0.696215,14.915742,0.561753,7.136027,0.569721,0.595618
3,ABBV,147.75,N/A,21.23,0.496016,18.24,0.930279,4.73,0.75996,11.590845,0.380478,7.677327,0.619522,0.637251
4,ABMD,263.93,N/A,56.47,0.922311,7.77,0.786853,11.20,0.956175,38.172997,0.962151,12.606233,0.864542,0.898406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,YUM,120.02,N/A,24.58,0.568725,-4.08,0.057769,5.13,0.794821,20.221361,0.792829,9.072440,0.695219,0.581873
498,ZBRA,297,N/A,31.92,0.760956,6.02,0.713147,2.58,0.463147,15.673768,0.605578,6.553558,0.494024,0.607371
499,ZBH,113.91,N/A,109.59,0.974104,2.06,0.320717,3.34,0.595618,18.487960,0.737052,5.717305,0.426295,0.610757
500,ZION,51.75,N/A,6.08,0.085657,1.51,0.201195,2.29,0.418327,4.752421,0.081673,2.218578,0.125498,0.18247


In [31]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

In [32]:
portfolio_input()

Enter the value of your portfolio:1000000


In [33]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,PVH,53.39,374,4.10,0.045817,0.6580,0.075697,0.3786,0.035857,4.286152,0.069721,0.911496,0.033865,0.052191
1,AIG,58.9,339,3.86,0.043825,1.0037,0.101594,0.7557,0.113546,2.261680,0.015936,0.742970,0.023904,0.059761
2,GM,40.06,499,6.91,0.10757,0.8617,0.085657,0.3917,0.041833,1.954808,0.013944,1.564276,0.089641,0.067729
3,BA,144.47,138,-10.21,0.025896,-4.9100,0.055777,1.4200,0.255976,-24.455531,0.003984,-151.675479,0.001992,0.068725
4,F,13.47,1484,4.77,0.054781,1.2000,0.13247,0.3565,0.02988,3.484300,0.035857,1.805600,0.099602,0.070518
5,C,46.17,433,5.95,0.081673,0.5092,0.069721,1.1200,0.187251,3.538735,0.037849,1.085705,0.051793,0.085657
6,HPQ,29.43,679,4.96,0.061753,-12.5900,0.041833,0.4376,0.051793,5.906970,0.123506,2.648896,0.163347,0.088446
7,PHM,40.97,488,4.20,0.049801,1.1700,0.125498,0.6040,0.077689,3.576671,0.039841,2.506307,0.153386,0.089243
8,UAA,7.67,2607,9.93,0.167331,1.4300,0.183267,0.5127,0.065737,2.837100,0.01992,0.661370,0.013944,0.09004
9,COF,107.9,185,4.91,0.059761,0.7914,0.081673,1.2500,0.22012,3.299354,0.027888,1.273694,0.069721,0.091833


In [41]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [42]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [43]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


In [44]:
writer.save()

/var/folders/sy/r631gy8n4r92m2pzw9tynp8w0000gn/T/ipykernel_10469/934276808.py:1: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()
